# Async Exercise: News Framing Dataset

In [2]:
import os
import pandas as pd
import numpy as np
import json
import re
import gensim
import nltk
import requests
import datetime

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from datetime import datetime

## Job Portal - WORKBANK

In [480]:
WORKBANK_JOB_TITLE = []
WORKBANK_JOB_CATEGORY = []
WORKBANK_JOB_COMPANY = []
WORKBANK_JOB_DATE = []
WORKBANK_JOB_LOCATION = []
WORKBANK_JOB_STATUS = []
WORKBANK_JOB_SALARY = []
WORKBANK_JOB_EDUCATION = []
WORKBANK_JOB_DESCRIPTION = []

In [465]:
#Information Communications Technology
IT_WORKBANK_URL = 'https://www.workbank.com/job/information-communications-technology-job-openings?wb_q='
IT_WORKBANK = requests.get(IT_WORKBANK_URL)

b'<!doctype html><html lang="en"><head><meta charset="utf-8"><script async src="https://www.googletagmanager.com/gtag/js?id=GTM-TL4H9C5"></script><script> window.dataLayer = window.dataLayer || []; function gtag(){dataLayer.push(arguments);} gtag(\'js\', new Date()); gtag(\'config\', \'UA-120994141-1\'); </script><title> Job hiring in Information Communications Technology | Find work @ Workbank </title><meta name="description" content="On a job search? You&#039;ve found the right career website! Find jobs by industry. Sign up for free and find work at Workbank today!"><meta name="og:type" content="website"/><meta name="og:title" content="Find careers in Information Communications Technology. Sign up now!"/><meta name="og:url" content="https://www.workbank.com/job/information-communications-technology-job-openings"/><meta name="og:description" content="See full list of careers in Information Communications Technology. Find work at Workbank today!"/><meta name="twitter:card" content="sum

In [466]:
IT_WORKBANK_soup = BeautifulSoup(IT_WORKBANK.content, 'html.parser')

In [467]:
IT_WORKBANK_PP = IT_WORKBANK_soup.find_all('select',{'class':'wb-pagination-select'})

[<select class="wb-pagination-select" id="page_no" name="page_no"><option data-current-page="1" data-goto="?page=1" selected="" value="1">1</option><option data-current-page="" data-goto="?page=2" value="2">2</option><option data-current-page="" data-goto="?page=3" value="3">3</option><option data-current-page="" data-goto="?page=4" value="4">4</option></select>]

In [468]:
if (len(IT_WORKBANK_PP)!=0):
    IT_WORKBANK_NUMPAGES=len(IT_WORKBANK_PP[0].select("option"))
else:
    IT_WORKBANK_NUMPAGES=0

4

In [469]:
IT_WORKBANK_PAGES=[]
if (IT_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        IT_WORKBANK_PAGES.append('https://www.workbank.com/job/information-communications-technology-job-openings?page='+ str(i))
else:
    IT_WORKBANK_PAGES.append('https://www.workbank.com/job/information-communications-technology-job-openings?page=1')

https://www.workbank.com/job/information-communications-technology-job-openings?page=1
https://www.workbank.com/job/information-communications-technology-job-openings?page=2
https://www.workbank.com/job/information-communications-technology-job-openings?page=3
https://www.workbank.com/job/information-communications-technology-job-openings?page=4


In [481]:
for i in range(len(IT_WORKBANK_PAGES)):
    IT_WORKBANK_URLs = IT_WORKBANK_PAGES[i]
    IT_WORKBANK_PAGE = requests.get(IT_WORKBANK_URLs)
    IT_WORKBANK_PAGE_soup = BeautifulSoup(IT_WORKBANK_PAGE.content, 'html.parser')
    IT_WORKBANK_JOBS = IT_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    IT_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(IT_WORKBANK_JOBS))
    IT_WORKBANK_JOB_DATEs = IT_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(IT_WORKBANK_JOB_URLs)):
        IT_WORKBANK_JOB_PAGE = requests.get(IT_WORKBANK_JOB_URLs[j])
        IT_WORKBANK_JOB_PAGE_soup = BeautifulSoup(IT_WORKBANK_JOB_PAGE.content, 'html.parser')
        IT_WORKBANK_JOB_PAGE_INFO1 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        IT_WORKBANK_JOB_TITLE = IT_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        IT_WORKBANK_JOB_COMPANY = IT_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        IT_WORKBANK_JOB_SALARY = IT_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        IT_WORKBANK_JOB_DATEPOSTED = IT_WORKBANK_JOB_DATEs[j].text.strip()
        IT_WORKBANK_JOB_LOCATION = IT_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        IT_WORKBANK_JOB_INFO2 = IT_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        IT_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(IT_WORKBANK_JOB_INFO2))[0]
        IT_WORKBANK_JOB_CATEGORY = "Information and Communications Technology"
        IT_WORKBANK_JOB_INFO3 = IT_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        IT_WORKBANK_JOB_DESCRIPTION = IT_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(IT_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(IT_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(IT_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(IT_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(IT_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(IT_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(IT_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(IT_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(IT_WORKBANK_JOB_DESCRIPTION)

In [472]:
#Construction
CONSTRUCTION_WORKBANK_URL = 'https://www.workbank.com/job/construction-job-openings?wb_q='
CONSTRUCTION_WORKBANK = requests.get(CONSTRUCTION_WORKBANK_URL)

b'<!doctype html><html lang="en"><head><meta charset="utf-8"><script async src="https://www.googletagmanager.com/gtag/js?id=GTM-TL4H9C5"></script><script> window.dataLayer = window.dataLayer || []; function gtag(){dataLayer.push(arguments);} gtag(\'js\', new Date()); gtag(\'config\', \'UA-120994141-1\'); </script><title> Job hiring in Construction | Find work @ Workbank </title><meta name="description" content="On a job search? You&#039;ve found the right career website! Find jobs by industry. Sign up for free and find work at Workbank today!"><meta name="og:type" content="website"/><meta name="og:title" content="Find careers in Construction. Sign up now!"/><meta name="og:url" content="https://www.workbank.com/job/construction-job-openings"/><meta name="og:description" content="See full list of careers in Construction. Find work at Workbank today!"/><meta name="twitter:card" content="summary" /><meta name="twitter:site" content="@WorkbankPh" /><meta name="twitter:title" content="Find c

In [473]:
CONSTRUCTION_WORKBANK_soup = BeautifulSoup(CONSTRUCTION_WORKBANK.content, 'html.parser')

In [474]:
CONSTRUCTION_WORKBANK_PP = CONSTRUCTION_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})

[]

In [475]:
if (len(CONSTRUCTION_WORKBANK_PP)!=0):
    CONSTRUCTION_WORKBANK_NUMPAGES=len(CONSTRUCTION_WORKBANK_PP[0].select("option"))
else:
    CONSTRUCTION_WORKBANK_NUMPAGES=0

0

In [476]:
CONSTRUCTION_WORKBANK_PAGES=[]
if (CONSTRUCTION_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        CONSTRUCTION_WORKBANK_PAGES.append('https://www.workbank.com/job/construction-job-openings?page='+ str(i))
else:
    CONSTRUCTION_WORKBANK_PAGES.append('https://www.workbank.com/job/construction-job-openings?page=1')

https://www.workbank.com/job/construction-job-openings?page=1


In [482]:
for i in range(len(CONSTRUCTION_WORKBANK_PAGES)):
    CONSTRUCTION_WORKBANK_URLs = CONSTRUCTION_WORKBANK_PAGES[i]
    CONSTRUCTION_WORKBANK_PAGE = requests.get(CONSTRUCTION_WORKBANK_URLs)
    CONSTRUCTION_WORKBANK_PAGE_soup = BeautifulSoup(CONSTRUCTION_WORKBANK_PAGE.content, 'html.parser')
    CONSTRUCTION_WORKBANK_JOBS = CONSTRUCTION_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    CONSTRUCTION_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(CONSTRUCTION_WORKBANK_JOBS))
    CONSTRUCTION_WORKBANK_JOB_DATEs = CONSTRUCTION_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(CONSTRUCTION_WORKBANK_JOB_URLs)):
        CONSTRUCTION_WORKBANK_JOB_PAGE = requests.get(CONSTRUCTION_WORKBANK_JOB_URLs[j])
        CONSTRUCTION_WORKBANK_JOB_PAGE_soup = BeautifulSoup(CONSTRUCTION_WORKBANK_JOB_PAGE.content, 'html.parser')
        CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1 = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        CONSTRUCTION_WORKBANK_JOB_TITLE = CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        CONSTRUCTION_WORKBANK_JOB_COMPANY = CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        CONSTRUCTION_WORKBANK_JOB_SALARY = CONSTRUCTION_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        CONSTRUCTION_WORKBANK_JOB_DATEPOSTED = CONSTRUCTION_WORKBANK_JOB_DATEs[j].text.strip()
        CONSTRUCTION_WORKBANK_JOB_LOCATION = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        CONSTRUCTION_WORKBANK_JOB_INFO2 = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        CONSTRUCTION_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(CONSTRUCTION_WORKBANK_JOB_INFO2))[0]
        CONSTRUCTION_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(CONSTRUCTION_WORKBANK_JOB_INFO2))[0]
        CONSTRUCTION_WORKBANK_JOB_CATEGORY = "Construction"
        CONSTRUCTION_WORKBANK_JOB_INFO3 = CONSTRUCTION_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        CONSTRUCTION_WORKBANK_JOB_DESCRIPTION = CONSTRUCTION_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(CONSTRUCTION_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(CONSTRUCTION_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(CONSTRUCTION_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(CONSTRUCTION_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(CONSTRUCTION_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(CONSTRUCTION_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(CONSTRUCTION_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(CONSTRUCTION_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(CONSTRUCTION_WORKBANK_JOB_DESCRIPTION)

In [485]:
#Design and Architecture
ARCHITECTURE_WORKBANK_URL = 'https://www.workbank.com/job/design-architecture-job-openings?wb_q='
ARCHITECTURE_WORKBANK = requests.get(ARCHITECTURE_WORKBANK_URL)
ARCHITECTURE_WORKBANK_soup = BeautifulSoup(ARCHITECTURE_WORKBANK.content, 'html.parser')
ARCHITECTURE_WORKBANK_PP = ARCHITECTURE_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(ARCHITECTURE_WORKBANK_PP)!=0):
    ARCHITECTURE_WORKBANK_NUMPAGES=len(ARCHITECTURE_WORKBANK_PP[0].select("option"))
else:
    ARCHITECTURE_WORKBANK_NUMPAGES=0
ARCHITECTURE_WORKBANK_PAGES=[]
if (ARCHITECTURE_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        ARCHITECTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/design-architecture-job-openings?page='+ str(i))
else:
    ARCHITECTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/design-architecture-job-openings?page=1')
for i in range(len(ARCHITECTURE_WORKBANK_PAGES)):
    ARCHITECTURE_WORKBANK_URLs = ARCHITECTURE_WORKBANK_PAGES[i]
    ARCHITECTURE_WORKBANK_PAGE = requests.get(ARCHITECTURE_WORKBANK_URLs)
    ARCHITECTURE_WORKBANK_PAGE_soup = BeautifulSoup(ARCHITECTURE_WORKBANK_PAGE.content, 'html.parser')
    ARCHITECTURE_WORKBANK_JOBS = ARCHITECTURE_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    ARCHITECTURE_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(ARCHITECTURE_WORKBANK_JOBS))
    ARCHITECTURE_WORKBANK_JOB_DATEs = ARCHITECTURE_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(ARCHITECTURE_WORKBANK_JOB_URLs)):
        ARCHITECTURE_WORKBANK_JOB_PAGE = requests.get(ARCHITECTURE_WORKBANK_JOB_URLs[j])
        ARCHITECTURE_WORKBANK_JOB_PAGE_soup = BeautifulSoup(ARCHITECTURE_WORKBANK_JOB_PAGE.content, 'html.parser')
        ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1 = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        ARCHITECTURE_WORKBANK_JOB_TITLE = ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        ARCHITECTURE_WORKBANK_JOB_COMPANY = ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        ARCHITECTURE_WORKBANK_JOB_SALARY = ARCHITECTURE_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        ARCHITECTURE_WORKBANK_JOB_DATEPOSTED = ARCHITECTURE_WORKBANK_JOB_DATEs[j].text.strip()
        ARCHITECTURE_WORKBANK_JOB_LOCATION = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        ARCHITECTURE_WORKBANK_JOB_INFO2 = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        ARCHITECTURE_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(ARCHITECTURE_WORKBANK_JOB_INFO2))[0]
        ARCHITECTURE_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(ARCHITECTURE_WORKBANK_JOB_INFO2))[0]
        ARCHITECTURE_WORKBANK_JOB_CATEGORY = "Design and Architecture"
        ARCHITECTURE_WORKBANK_JOB_INFO3 = ARCHITECTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        ARCHITECTURE_WORKBANK_JOB_DESCRIPTION = ARCHITECTURE_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(ARCHITECTURE_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(ARCHITECTURE_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(ARCHITECTURE_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(ARCHITECTURE_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(ARCHITECTURE_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(ARCHITECTURE_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(ARCHITECTURE_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(ARCHITECTURE_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(ARCHITECTURE_WORKBANK_JOB_DESCRIPTION)

In [486]:
#Agriculture and Wildlife Conservation
AGRICULTURE_WORKBANK_URL = 'https://www.workbank.com/job/agriculture-wildlife-conservation-job-openings?wb_q='
AGRICULTURE_WORKBANK = requests.get(AGRICULTURE_WORKBANK_URL)
AGRICULTURE_WORKBANK_soup = BeautifulSoup(AGRICULTURE_WORKBANK.content, 'html.parser')
AGRICULTURE_WORKBANK_PP = AGRICULTURE_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(AGRICULTURE_WORKBANK_PP)!=0):
    AGRICULTURE_WORKBANK_NUMPAGES=len(AGRICULTURE_WORKBANK_PP[0].select("option"))
else:
    AGRICULTURE_WORKBANK_NUMPAGES=0
AGRICULTURE_WORKBANK_PAGES=[]
if (AGRICULTURE_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        AGRICULTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/agriculture-wildlife-conservation-job-openings?page='+ str(i))
else:
    AGRICULTURE_WORKBANK_PAGES.append('https://www.workbank.com/job/agriculture-wildlife-conservation-job-openings?page=1')
for i in range(len(AGRICULTURE_WORKBANK_PAGES)):
    AGRICULTURE_WORKBANK_URLs = AGRICULTURE_WORKBANK_PAGES[i]
    AGRICULTURE_WORKBANK_PAGE = requests.get(AGRICULTURE_WORKBANK_URLs)
    AGRICULTURE_WORKBANK_PAGE_soup = BeautifulSoup(AGRICULTURE_WORKBANK_PAGE.content, 'html.parser')
    AGRICULTURE_WORKBANK_JOBS = AGRICULTURE_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    AGRICULTURE_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(AGRICULTURE_WORKBANK_JOBS))
    AGRICULTURE_WORKBANK_JOB_DATEs = AGRICULTURE_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(AGRICULTURE_WORKBANK_JOB_URLs)):
        AGRICULTURE_WORKBANK_JOB_PAGE = requests.get(AGRICULTURE_WORKBANK_JOB_URLs[j])
        AGRICULTURE_WORKBANK_JOB_PAGE_soup = BeautifulSoup(AGRICULTURE_WORKBANK_JOB_PAGE.content, 'html.parser')
        AGRICULTURE_WORKBANK_JOB_PAGE_INFO1 = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        AGRICULTURE_WORKBANK_JOB_TITLE = AGRICULTURE_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        AGRICULTURE_WORKBANK_JOB_COMPANY = AGRICULTURE_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        AGRICULTURE_WORKBANK_JOB_SALARY = AGRICULTURE_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        AGRICULTURE_WORKBANK_JOB_DATEPOSTED = AGRICULTURE_WORKBANK_JOB_DATEs[j].text.strip()
        AGRICULTURE_WORKBANK_JOB_LOCATION = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        AGRICULTURE_WORKBANK_JOB_INFO2 = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        AGRICULTURE_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(AGRICULTURE_WORKBANK_JOB_INFO2))[0]
        AGRICULTURE_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(AGRICULTURE_WORKBANK_JOB_INFO2))[0]
        AGRICULTURE_WORKBANK_JOB_CATEGORY = "Agriculture and Wildlife Conservation"
        AGRICULTURE_WORKBANK_JOB_INFO3 = AGRICULTURE_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        AGRICULTURE_WORKBANK_JOB_DESCRIPTION = AGRICULTURE_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(AGRICULTURE_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(AGRICULTURE_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(AGRICULTURE_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(AGRICULTURE_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(AGRICULTURE_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(AGRICULTURE_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(AGRICULTURE_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(AGRICULTURE_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(AGRICULTURE_WORKBANK_JOB_DESCRIPTION)

In [487]:
#Environmental and Health Safety
SAFETY_WORKBANK_URL = 'https://www.workbank.com/job/environmental-health-safety-job-openings?wb_q='
SAFETY_WORKBANK = requests.get(SAFETY_WORKBANK_URL)
SAFETY_WORKBANK_soup = BeautifulSoup(SAFETY_WORKBANK.content, 'html.parser')
SAFETY_WORKBANK_PP = SAFETY_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(SAFETY_WORKBANK_PP)!=0):
    SAFETY_WORKBANK_NUMPAGES=len(SAFETY_WORKBANK_PP[0].select("option"))
else:
    SAFETY_WORKBANK_NUMPAGES=0
SAFETY_WORKBANK_PAGES=[]
if (SAFETY_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        SAFETY_WORKBANK_PAGES.append('https://www.workbank.com/job/environmental-health-safety-job-openings?page='+ str(i))
else:
    SAFETY_WORKBANK_PAGES.append('https://www.workbank.com/job/environmental-health-safety-job-openings?page=1')
for i in range(len(SAFETY_WORKBANK_PAGES)):
    SAFETY_WORKBANK_URLs = SAFETY_WORKBANK_PAGES[i]
    SAFETY_WORKBANK_PAGE = requests.get(SAFETY_WORKBANK_URLs)
    SAFETY_WORKBANK_PAGE_soup = BeautifulSoup(SAFETY_WORKBANK_PAGE.content, 'html.parser')
    SAFETY_WORKBANK_JOBS = SAFETY_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    SAFETY_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(SAFETY_WORKBANK_JOBS))
    SAFETY_WORKBANK_JOB_DATEs = SAFETY_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(SAFETY_WORKBANK_JOB_URLs)):
        SAFETY_WORKBANK_JOB_PAGE = requests.get(SAFETY_WORKBANK_JOB_URLs[j])
        SAFETY_WORKBANK_JOB_PAGE_soup = BeautifulSoup(SAFETY_WORKBANK_JOB_PAGE.content, 'html.parser')
        SAFETY_WORKBANK_JOB_PAGE_INFO1 = SAFETY_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        SAFETY_WORKBANK_JOB_TITLE = SAFETY_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        SAFETY_WORKBANK_JOB_COMPANY = SAFETY_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        SAFETY_WORKBANK_JOB_SALARY = SAFETY_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        SAFETY_WORKBANK_JOB_DATEPOSTED = SAFETY_WORKBANK_JOB_DATEs[j].text.strip()
        SAFETY_WORKBANK_JOB_LOCATION = SAFETY_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        SAFETY_WORKBANK_JOB_INFO2 = SAFETY_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        SAFETY_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(SAFETY_WORKBANK_JOB_INFO2))[0]
        SAFETY_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(SAFETY_WORKBANK_JOB_INFO2))[0]
        SAFETY_WORKBANK_JOB_CATEGORY = "Environmental and Health Safety"
        SAFETY_WORKBANK_JOB_INFO3 = SAFETY_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        SAFETY_WORKBANK_JOB_DESCRIPTION = SAFETY_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(SAFETY_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(SAFETY_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(SAFETY_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(SAFETY_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(SAFETY_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(SAFETY_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(SAFETY_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(SAFETY_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(SAFETY_WORKBANK_JOB_DESCRIPTION)

In [488]:
#Medical and Healthcare
HEALTH_WORKBANK_URL = 'https://www.workbank.com/job/medical-healthcare-job-openings?wb_q='
HEALTH_WORKBANK = requests.get(HEALTH_WORKBANK_URL)
HEALTH_WORKBANK_soup = BeautifulSoup(HEALTH_WORKBANK.content, 'html.parser')
HEALTH_WORKBANK_PP = HEALTH_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(HEALTH_WORKBANK_PP)!=0):
    HEALTH_WORKBANK_NUMPAGES=len(HEALTH_WORKBANK_PP[0].select("option"))
else:
    HEALTH_WORKBANK_NUMPAGES=0
HEALTH_WORKBANK_PAGES=[]
if (HEALTH_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        HEALTH_WORKBANK_PAGES.append('https://www.workbank.com/job/medical-healthcare-job-openings?page='+ str(i))
else:
    HEALTH_WORKBANK_PAGES.append('https://www.workbank.com/job/medical-healthcare-job-openings?page=1')
for i in range(len(HEALTH_WORKBANK_PAGES)):
    HEALTH_WORKBANK_URLs = HEALTH_WORKBANK_PAGES[i]
    HEALTH_WORKBANK_PAGE = requests.get(HEALTH_WORKBANK_URLs)
    HEALTH_WORKBANK_PAGE_soup = BeautifulSoup(HEALTH_WORKBANK_PAGE.content, 'html.parser')
    HEALTH_WORKBANK_JOBS = HEALTH_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    HEALTH_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(HEALTH_WORKBANK_JOBS))
    HEALTH_WORKBANK_JOB_DATEs = HEALTH_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(HEALTH_WORKBANK_JOB_URLs)):
        HEALTH_WORKBANK_JOB_PAGE = requests.get(HEALTH_WORKBANK_JOB_URLs[j])
        HEALTH_WORKBANK_JOB_PAGE_soup = BeautifulSoup(HEALTH_WORKBANK_JOB_PAGE.content, 'html.parser')
        HEALTH_WORKBANK_JOB_PAGE_INFO1 = HEALTH_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        HEALTH_WORKBANK_JOB_TITLE = HEALTH_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        HEALTH_WORKBANK_JOB_COMPANY = HEALTH_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        HEALTH_WORKBANK_JOB_SALARY = HEALTH_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        HEALTH_WORKBANK_JOB_DATEPOSTED = HEALTH_WORKBANK_JOB_DATEs[j].text.strip()
        HEALTH_WORKBANK_JOB_LOCATION = HEALTH_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        HEALTH_WORKBANK_JOB_INFO2 = HEALTH_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        HEALTH_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(HEALTH_WORKBANK_JOB_INFO2))[0]
        HEALTH_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(HEALTH_WORKBANK_JOB_INFO2))[0]
        HEALTH_WORKBANK_JOB_CATEGORY = "Medical and Healthcare"
        HEALTH_WORKBANK_JOB_INFO3 = HEALTH_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        HEALTH_WORKBANK_JOB_DESCRIPTION = HEALTH_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(HEALTH_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(HEALTH_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(HEALTH_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(HEALTH_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(HEALTH_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(HEALTH_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(HEALTH_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(HEALTH_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(HEALTH_WORKBANK_JOB_DESCRIPTION)

In [489]:
#Sciences
SCIENCES_WORKBANK_URL = 'https://www.workbank.com/job/sciences-job-openings?wb_q='
SCIENCES_WORKBANK = requests.get(SCIENCES_WORKBANK_URL)
SCIENCES_WORKBANK_soup = BeautifulSoup(SCIENCES_WORKBANK.content, 'html.parser')
SCIENCES_WORKBANK_PP = SCIENCES_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(SCIENCES_WORKBANK_PP)!=0):
    SCIENCES_WORKBANK_NUMPAGES=len(SCIENCES_WORKBANK_PP[0].select("option"))
else:
    SCIENCES_WORKBANK_NUMPAGES=0
SCIENCES_WORKBANK_PAGES=[]
if (SCIENCES_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        SCIENCES_WORKBANK_PAGES.append('https://www.workbank.com/job/sciences-job-openings?page='+ str(i))
else:
    SCIENCES_WORKBANK_PAGES.append('https://www.workbank.com/job/sciences-job-openings?page=1')
for i in range(len(SCIENCES_WORKBANK_PAGES)):
    SCIENCES_WORKBANK_URLs = SCIENCES_WORKBANK_PAGES[i]
    SCIENCES_WORKBANK_PAGE = requests.get(SCIENCES_WORKBANK_URLs)
    SCIENCES_WORKBANK_PAGE_soup = BeautifulSoup(SCIENCES_WORKBANK_PAGE.content, 'html.parser')
    SCIENCES_WORKBANK_JOBS = SCIENCES_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    SCIENCES_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(SCIENCES_WORKBANK_JOBS))
    SCIENCES_WORKBANK_JOB_DATEs = SCIENCES_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(SCIENCES_WORKBANK_JOB_URLs)):
        SCIENCES_WORKBANK_JOB_PAGE = requests.get(SCIENCES_WORKBANK_JOB_URLs[j])
        SCIENCES_WORKBANK_JOB_PAGE_soup = BeautifulSoup(SCIENCES_WORKBANK_JOB_PAGE.content, 'html.parser')
        SCIENCES_WORKBANK_JOB_PAGE_INFO1 = SCIENCES_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        SCIENCES_WORKBANK_JOB_TITLE = SCIENCES_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        SCIENCES_WORKBANK_JOB_COMPANY = SCIENCES_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        SCIENCES_WORKBANK_JOB_SALARY = SCIENCES_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        SCIENCES_WORKBANK_JOB_DATEPOSTED = SCIENCES_WORKBANK_JOB_DATEs[j].text.strip()
        SCIENCES_WORKBANK_JOB_LOCATION = SCIENCES_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        SCIENCES_WORKBANK_JOB_INFO2 = SCIENCES_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        SCIENCES_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(SCIENCES_WORKBANK_JOB_INFO2))[0]
        SCIENCES_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(SCIENCES_WORKBANK_JOB_INFO2))[0]
        SCIENCES_WORKBANK_JOB_CATEGORY = "Sciences"
        SCIENCES_WORKBANK_JOB_INFO3 = SCIENCES_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        SCIENCES_WORKBANK_JOB_DESCRIPTION = SCIENCES_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(SCIENCES_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(SCIENCES_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(SCIENCES_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(SCIENCES_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(SCIENCES_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(SCIENCES_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(SCIENCES_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(SCIENCES_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(SCIENCES_WORKBANK_JOB_DESCRIPTION)

In [490]:
#Actuarial
ACTUARIAL_WORKBANK_URL = 'https://www.workbank.com/job/hiring-actuarial'
ACTUARIAL_WORKBANK = requests.get(ACTUARIAL_WORKBANK_URL)
ACTUARIAL_WORKBANK_soup = BeautifulSoup(ACTUARIAL_WORKBANK.content, 'html.parser')
ACTUARIAL_WORKBANK_PP = ACTUARIAL_WORKBANK_soup.find_all('div',{'class':'wb-pagination'})
if (len(ACTUARIAL_WORKBANK_PP)!=0):
    ACTUARIAL_WORKBANK_NUMPAGES=len(ACTUARIAL_WORKBANK_PP[0].select("option"))
else:
    ACTUARIAL_WORKBANK_NUMPAGES=0
ACTUARIAL_WORKBANK_PAGES=[]
if (ACTUARIAL_WORKBANK_NUMPAGES!=0):
    for i in range(1,IT_WORKBANK_NUMPAGES+1):
        ACTUARIAL_WORKBANK_PAGES.append('https://www.workbank.com/job/hiring-actuarial?page='+ str(i))
else:
    ACTUARIAL_WORKBANK_PAGES.append('https://www.workbank.com/job/hiring-actuarial?page=1')
for i in range(len(ACTUARIAL_WORKBANK_PAGES)):
    ACTUARIAL_WORKBANK_URLs = ACTUARIAL_WORKBANK_PAGES[i]
    ACTUARIAL_WORKBANK_PAGE = requests.get(ACTUARIAL_WORKBANK_URLs)
    ACTUARIAL_WORKBANK_PAGE_soup = BeautifulSoup(ACTUARIAL_WORKBANK_PAGE.content, 'html.parser')
    ACTUARIAL_WORKBANK_JOBS = ACTUARIAL_WORKBANK_PAGE_soup.find_all('a',{'class':'clearfix'})
    ACTUARIAL_WORKBANK_JOB_URLs = re.findall(r'(?s)(?<=href=").*?(?="><h5)',str(ACTUARIAL_WORKBANK_JOBS))
    ACTUARIAL_WORKBANK_JOB_DATEs = ACTUARIAL_WORKBANK_PAGE_soup.find_all('p',{'class':'publish-date-card mt-1 text-left mb-0'})
    for j in range(len(ACTUARIAL_WORKBANK_JOB_URLs)):
        ACTUARIAL_WORKBANK_JOB_PAGE = requests.get(ACTUARIAL_WORKBANK_JOB_URLs[j])
        ACTUARIAL_WORKBANK_JOB_PAGE_soup = BeautifulSoup(ACTUARIAL_WORKBANK_JOB_PAGE.content, 'html.parser')
        ACTUARIAL_WORKBANK_JOB_PAGE_INFO1 = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('article',{'class':'job-ad-text-center pl-3'})
        ACTUARIAL_WORKBANK_JOB_TITLE = ACTUARIAL_WORKBANK_JOB_PAGE_INFO1.contents[0].text.strip()
        ACTUARIAL_WORKBANK_JOB_COMPANY = ACTUARIAL_WORKBANK_JOB_PAGE_INFO1.contents[1].text.strip()
        ACTUARIAL_WORKBANK_JOB_SALARY = ACTUARIAL_WORKBANK_JOB_PAGE_INFO1.contents[4].text.strip()
        ACTUARIAL_WORKBANK_JOB_DATEPOSTED = ACTUARIAL_WORKBANK_JOB_DATEs[j].text.strip()
        ACTUARIAL_WORKBANK_JOB_LOCATION = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('a',{'class':'cls-links'}).text.strip()
        ACTUARIAL_WORKBANK_JOB_INFO2 = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('ul',{'class':'job-ad-des-ul mb-0'})
        ACTUARIAL_WORKBANK_JOB_STATUS = re.findall(r'(?s)(?<=Job Type</h5><p>).*?(?=</p>)',str(ACTUARIAL_WORKBANK_JOB_INFO2))[0]
        ACTUARIAL_WORKBANK_JOB_EDUCATION = re.findall(r'(?s)(?<=Educational Attainment</h5><p>).*?(?=</p>)',str(ACTUARIAL_WORKBANK_JOB_INFO2))[0]
        ACTUARIAL_WORKBANK_JOB_CATEGORY = "Actuarial"
        ACTUARIAL_WORKBANK_JOB_INFO3 = ACTUARIAL_WORKBANK_JOB_PAGE_soup.find('article',{'class':'pl-4 pr-4 pb-0 pt-4'})
        ACTUARIAL_WORKBANK_JOB_DESCRIPTION = ACTUARIAL_WORKBANK_JOB_INFO3.contents[1].getText(separator=u' ')
        WORKBANK_JOB_TITLE.append(ACTUARIAL_WORKBANK_JOB_TITLE)
        WORKBANK_JOB_CATEGORY.append(ACTUARIAL_WORKBANK_JOB_CATEGORY)
        WORKBANK_JOB_COMPANY.append(ACTUARIAL_WORKBANK_JOB_COMPANY)
        WORKBANK_JOB_DATE.append(ACTUARIAL_WORKBANK_JOB_DATEPOSTED)
        WORKBANK_JOB_LOCATION.append(ACTUARIAL_WORKBANK_JOB_LOCATION)
        WORKBANK_JOB_STATUS.append(ACTUARIAL_WORKBANK_JOB_STATUS)
        WORKBANK_JOB_SALARY.append(ACTUARIAL_WORKBANK_JOB_SALARY)
        WORKBANK_JOB_EDUCATION.append(ACTUARIAL_WORKBANK_JOB_EDUCATION)
        WORKBANK_JOB_DESCRIPTION.append(ACTUARIAL_WORKBANK_JOB_DESCRIPTION)

In [491]:
workbank={'Job Title': WORKBANK_JOB_TITLE, 'Category': WORKBANK_JOB_CATEGORY, 'Company': WORKBANK_JOB_COMPANY, 'Date Posted': WORKBANK_JOB_DATE, 
          'Location': WORKBANK_JOB_LOCATION, 'Status': WORKBANK_JOB_STATUS, 'Salary': WORKBANK_JOB_SALARY, 'Education': WORKBANK_JOB_EDUCATION, 
          'Job Description': WORKBANK_JOB_DESCRIPTION}
workbank_df = pd.DataFrame(data=workbank)
workbank_df

,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Job Description
0,VAPT Specialist,Information and Communications Technology,"Indra Philippines, Inc.",Posted 15 hours ago,Work from Home,Contract,"PHP 70,000 - 80,000",Bachelor’s Degree,QUALIFICATIONS: -- At least 2 years working e...
1,Security Engineer,Information and Communications Technology,"Indra Philippines, Inc.",Posted 15 hours ago,Work from Home,Contract,"PHP 70,000 - 80,000",Bachelor’s Degree,GENERAL RESPONSIBILITIES - Ensures that secur...
2,Application Developer,Information and Communications Technology,Tangent Solutions Inc.,Posted 1 day ago,Pasig,Full Time,"PHP 35,000 - 40,000",Bachelor’s Degree,Responsibilities: • Develops applications...
3,Dynamics CRM And .Net Developer,Information and Communications Technology,"Indra Philippines, Inc.",Posted 3 days ago,Work from Home,Full Time,,Bachelor’s Degree,Indra Philippines is hiring for a Dynamics CRM...
4,QUALITY ANALYST,Information and Communications Technology,UNIVERSAL LEAF PHILS. INC.,Posted 4 days ago,Agoo,Full Time,,Bachelor’s Degree,The principal responsibility of a Software Qua...
...,...,...,...,...,...,...,...,...,...
77,PH Registered Nurse (PHRN),Medical and Healthcare,Hinduja Global Solutions,Posted 2 months ago,Quezon City,Full Time,"PHP 25,000 - 30,000",Bachelor’s Degree,MINIMUM QUALIFICATIONS: BS NURSING Graduate Mu...
78,US Registered Nurse (USRN),Medical and Healthcare,Hinduja Global Solutions,Posted 2 months ago,Quezon City,Full Time,,Bachelor’s Degree,US Registered Nurse (USRN) with 100k Retention...
79,Company Nurse,Medical and Healthcare,"Gardenia Bakeries Philippines, Inc.",Posted 2 months ago,Biñan City,Contract,,Bachelor’s Degree,Administer first aid treatments and provide me...
80,US Registered Nurse Analyst,Medical and Healthcare,"Accenture, Inc.",Posted 2 months ago,National Capital Region,Full Time,"PHP 35,000 - 40,000",Bachelor’s Degree,Go through a faster and more convenient recrui...


In [493]:
data = workbank_df.to_json(orient='records')
parsed = json.loads(data)
json.dumps(parsed, indent=4) 
with open('workbank.json', 'w') as json_file:
    json.dump(parsed, json_file)